# PancakeSwap Prediction Game

PCS has a cool WBNB prediction game, where every 5 minutes you get to bid on whether the price will be higher or lower after 5 more minutes. If you're right, you win a multiple of your bid (minus fees). Let's see if we can win a this, and what winning would actually mean.

In [1]:
# Standard library modules
import os
import sys
import math
import time
import json
import random
import numbers
import datetime

# Extra installed modules
import web3
import tqdm
import numpy  as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import mplfinance as mpf
%matplotlib widget

# My own functions for analysis of BSC data
import bsc_analysis

In [2]:
# For when I change the bsc_analysis module
import importlib
importlib.reload(bsc_analysis)

<module 'bsc_analysis' from '/home/jovyan/repos/bsc-learning/bsc_analysis.py'>

## PancakeSwap Prediction Data

Implement a class that obtains data about past Prediction games on PancakeSwap, can save/load as JSON, and can update on demand.


In [3]:
class PredictionData(dict):
    storage_file      = 'pcs_prediction.json'
    pks_contract_addr = web3.Web3.toChecksumAddress('0x18b2a687610328590bc8f2e5fedde3b582a49cda')
    bsc_data_url      = 'https://bsc-dataseed.binance.org/'
    bsc_api_url       = 'https://api.bscscan.com/api'
    connection        = None
    pks_contract      = None
    data              = None
    n_rounds          = 0
    
    def __init__(self, filename= None):
        self.data = {}
        if filename is not None:
            self.storage_file = filename
        self.load()
        self.connect()
        self.load_contract()
        self.update()
        return
    
    def __len__(self):
        return self.n_rounds
    
    def __getitem__(self, key):
        if isinstance(key, numbers.Number):
            return self.data[str(key)]
        return self.data[key]
    
    def save(self):
        if not self.data:
            return
        try:
            with open(self.storage_file, 'w') as outfile:
                json.dump(self.data, outfile)
                print(f'Saved {len(self.data)} rounds to storage file')
        except Exception as err:
            print(f'Unable to save storage file: {err}')
        return
    
    def load(self):
        try:
            with open(self.storage_file, 'r') as infile:
                self.data = json.load(infile)
                print(f'Loaded {len(self.data)} rounds from storage file')
        except Exception as err:
            print(f'Unable to load storage file: {err}')
        return
    
    def keys(self):
        if self.data is not None:
            return self.data.keys()
        return
    
    def items(self):
        if self.data is not None:
            return self.data.items()
        return
    
    def connect(self):
        connection = web3.Web3(web3.Web3.HTTPProvider(self.bsc_data_url))
        if not connection.isConnected():
            self.connection = None
            return False
        self.connection = connection
        print('Connected')
        return True
    
    def load_contract(self):
        abi_url           = f'{self.bsc_api_url}?module=contract&action=getabi&address={self.pks_contract_addr}'
        rr                = requests.get(url = abi_url)
        self.abi          = json.loads(rr.json()['result'])
        self.pks_contract = self.connection.eth.contract(address= self.pks_contract_addr, abi= self.abi)
        self.func_inputs  = {func['name']: func[ 'inputs'] for func in self.abi if 'name' in func and  'inputs' in func}
        self.func_outputs = {func['name']: func['outputs'] for func in self.abi if 'name' in func and 'outputs' in func}
        print('PKS Prediction Contract Loaded')
        return
    
    def get_round(self, index):
        func = self.pks_contract.functions.rounds(index) 
        rlist = func.call()
        return {spec['name']: value for value, spec in zip(rlist, self.func_outputs['rounds'])}
    
    def update(self):
        if self.data is None:
            self.data = {}
        self.n_rounds = self.pks_contract.functions.currentEpoch().call()
        n_pulled = 0
        n_saved  = 0
        for ii in tqdm.tqdm(range(self.n_rounds)):
            iis = str(ii)
            if iis not in self.data:
                self.data[iis] = self.get_round(ii)
                n_pulled += 1
                if n_pulled > 5000:
                    self.save()
                    n_saved += n_pulled
                    n_pulled = 0
        self.save()
        return

## Prediction Strategies

Implement various prediction strategies to be backtested against the data.

In [4]:
def strategy_biggest(round_info, ohlc_stats):
    if round_info['up payout'] > round_info['down payout']:
        return 'up'
    elif round_info['up payout'] < round_info['down payout']:
        return 'down'
    return None

def strategy_smallest(round_info, ohlc_stats):
    if round_info['up payout'] > round_info['down payout']:
        return 'down'
    elif round_info['up payout'] < round_info['down payout']:
        return 'up'
    return None

def strategy_bull(round_info, ohlc_stats):
    return 'up'

def strategy_bear(round_info, ohlc_stats):
    return 'down'

basic_setup = {
    'starting bnb': 1,
    'betting bnb' : 0.001,
    'winnings tax': 0.03,
    'betting fee' : 0.0006,
}

bold_setup = {
    'starting bnb': 4,
    'betting bnb' : 0.001,
    'winnings tax': 0.03,
    'betting fee' : 0.0006,
}

free_setup = {
    'starting bnb': 1,
    'betting bnb' : 0.001,
    'winnings tax': 0.03,
    'betting fee' : 0,
}

In [5]:
def strategy_ema_1(round_info, ohlc_stats, verbose= False):
    start = datetime.datetime.fromtimestamp(round_info['startTimestamp'])
    lock  = datetime.datetime.fromtimestamp(round_info['lockTimestamp'])
    close = datetime.datetime.fromtimestamp(round_info['closeTimestamp'])
    # Pull EMA data at 1 minute prior to the lock time
    pull  = (lock - datetime.timedelta(minutes= 1)).replace(second= 0, microsecond=0)
    
    if verbose:
        print(f'Round {round_info["epoch"]}: [{start}] - [{close}], locked at [{lock}], pull OHLC from [{pull}]')
    isopull = pull.isoformat()
    try:
        oo, hh, ll, cc, tt = ohlc = stats.data.loc[isopull].values[0]
        emas  = stats.emas.loc[isopull].todict()
        cross = stats.cross.loc[isopull].todict()
    except:
        return
        
    if verbose:
        print(f'OHLC: {oo} {hh} {ll} {cc} {tt}')
        print(emas)
        print(cross)
    
    # If the price keeps rising, take note
    try:
        if hh > emas['ema_high_10'] > emas['ema_high_20'] > emas['ema_high_60'] > emas['ema_high_120']:
            #if cross['xover_[high]_[ema_high_10]'] < 10:
            return 'up'
    except Exception as err:
        vat = lambda x: f'[{x}:{type(x)}]'
        print(f'{isopull}: ({vat(oo)}, {vat(hh)}, {vat(ll)}, {vat(cc)}, {vat(tt)}) {vat(emas["ema_high_10"])} {vat(emas["ema_high_20"])} {vat(emas["ema_high_60"])} {vat(emas["ema_high_120"])}')
        print(err)
    #if hh > highs[0] > highs[1] > highs[2] and ll > lows[0] > lows[1] > lows[2] and round_info['up payout'] > min(2.5, round_info['down payout']):
    #    return 'up'
    #if hh < highs[0] < highs[1] < highs[2] and ll < lows[0] < lows[1] < lows[2] and round_info['down payout'] > min(2.5, round_info['up payout']):
    #    return 'down'
    return None

basic_strategies = {
    'Always pick biggest payout' : strategy_biggest,
    'Always pick smallest payout': strategy_smallest,
    'Always pick up'             : strategy_bull,
    'Always pick down'           : strategy_bear,
    'Only pick obvious runs'     : strategy_ema_1,
}

In [6]:
def reference_strategy_generator(rate=0.7):
    def gen(round_info, ohlc_stats):
        win  = (round_info['closePrice'] > round_info['lockPrice']) and 'up' or 'down'
        lose = (round_info['closePrice'] > round_info['lockPrice']) and 'down' or 'up'
        coin = random.random()
        if coin < rate:
            return win
        return lose
    return gen

ref_strategies = {f'{pp:2d}% success rate': reference_strategy_generator(pp / 100.) for pp in range(80,101, 5)}

## Loading The Data

Get the PCS prediction data. Then get OHLCs for WBNB.

The bsc_analysis module includes an OHLCData class that obtains OHLC data from whatever source, can save/load as JSON, and can update on demand. There's also an OHLCStats class that calculates EMAs and crossing points. Both are thin wrappers around DataFrames.

In [7]:
pred = PredictionData()

Loaded 49521 rounds from storage file
Connected
PKS Prediction Contract Loaded


100%|██████████| 49533/49533 [00:00<00:00, 56685.47it/s] 


Saved 49533 rounds to storage file


In [8]:
ohlc = bsc_analysis.OHLCData(start_date='2021-07-01')
stats = bsc_analysis.OHLCStats(ohlc.data)

Loaded 244 days of OHLC from storage file
Retrieving data:


OHLC data [2022-03-01] pulled=   0 saved=   0 : 100%|██████████| 244/244 [00:01<00:00, 164.78it/s]


Saved 244 days of OHLC to storage file


## Pulling it Together

Load/update the prediction data, and then backtest a list of prediction strategies, presenting the results in tabular form.

In [9]:
def evaluate_strategies(setup, strategies, stats):
    for strategy in strategies:
        evaluator = strategies[strategy]
        n_played  = 0
        n_won     = 0
        n_lost    = 0
        n_zero    = 0
        n_skip    = 0
        n_iter    = 0
        balance   = setup['starting bnb']
        for rr in range(len(pred)):
            n_iter += 1
            if pred[rr]['totalAmount'] == 0 or pred[rr]['bullAmount'] == 0 or pred[rr]['bearAmount'] == 0:
                n_zero += 1
                continue

            # Calculate on win, payout on win, including our contrbution to the pool
            pred[rr]['up payout'  ] = (pred[rr]['totalAmount'] + setup['betting bnb']) / (pred[rr]['bullAmount'] + setup['betting bnb'])
            pred[rr]['down payout'] = (pred[rr]['totalAmount'] + setup['betting bnb']) / (pred[rr]['bearAmount'] + setup['betting bnb'])            
            bet = evaluator(pred[rr], stats)
            if bet:
                n_played   += 1
                balance    -= setup['betting bnb'] + setup['betting fee']
                went_up     = pred[rr]['closePrice'] > pred[rr]['lockPrice']
                if bet == 'up' and went_up:
                    n_won   += 1
                    balance += setup['betting bnb'] * pred[rr]['up payout'  ] * (1 - setup['winnings tax'])
                elif bet == 'down' and not went_up:
                    n_won   += 1
                    balance += setup['betting bnb'] * pred[rr]['down payout'] * (1 - setup['winnings tax'])
                else:
                    n_lost  += 1
                if balance < setup['betting bnb'] + setup['betting fee']:
                    break
            else:
                n_skip += 1
        pnl = 100 * (balance / setup['starting bnb'] - 1)
        judgement = (-0.05 < pnl < 0.05 and "BREAK-EVEN") or (pnl < -99. and "BANKRUPT") or (pnl < 0 and "LOSS") or "WIN"
        print(f'{strategy:30}: played={n_played:5} won={n_won:6} lost={n_lost:6} skip={n_skip:6} balance={balance:9.5f} pnl%={pnl:9.5f} {judgement}')

In [10]:
print('FREE SETUP, BASIC STRATEGIES')
evaluate_strategies(free_setup, basic_strategies, stats)
print('-'*80)
print('BASIC SETUP, BASIC STRATEGIES')
evaluate_strategies(basic_setup, basic_strategies, stats)
print('-'*80)
print('BOLD SETUP, BASIC STRATEGIES')
evaluate_strategies(bold_setup, basic_strategies, stats)
print('-'*80)
print('FREE SETUP, REFERENCE STRATEGIES')
evaluate_strategies(free_setup, ref_strategies, stats)
print('-'*80)
print('BASIC SETUP, REFERENCE STRATEGIES')
evaluate_strategies(basic_setup, ref_strategies, stats)
print('-'*80)
print('BOLD SETUP, REFERENCE STRATEGIES')
evaluate_strategies(bold_setup, ref_strategies, stats)

FREE SETUP, BASIC STRATEGIES
Always pick biggest payout    : played=49447 won= 22358 lost= 27089 skip=     0 balance=  2.56958 pnl%=156.95752 WIN
Always pick smallest payout   : played=13301 won=  7121 lost=  6180 skip=     0 balance=  0.00093 pnl%=-99.90667 BANKRUPT
Always pick up                : played=37667 won= 18940 lost= 18727 skip=     0 balance=  0.00004 pnl%=-99.99555 BANKRUPT
Always pick down              : played=49447 won= 24548 lost= 24899 skip=     0 balance=  0.71644 pnl%=-28.35635 LOSS
Only pick obvious runs        : played=    0 won=     0 lost=     0 skip= 49447 balance=  1.00000 pnl%=  0.00000 BREAK-EVEN
--------------------------------------------------------------------------------
BASIC SETUP, BASIC STRATEGIES
Always pick biggest payout    : played= 2069 won=   974 lost=  1095 skip=     0 balance=  0.00083 pnl%=-99.91704 BANKRUPT
Always pick smallest payout   : played= 1393 won=   733 lost=   660 skip=     0 balance=  0.00060 pnl%=-99.94022 BANKRUPT
Always pick u

Conclusion: We need over 80% success rate to make anything at PCS prediction. I'm nowhere near that level, and if I were, there are easier ways to make money.